In [60]:
import pandas as pd

In [61]:
cluster_data_fp = "../../data/clusters/voter_clusters.csv"
cluster_df = pd.read_csv(cluster_data_fp)

In [62]:
cluster_df.head()

,voters,Government operations and politics_ Government operations and politics_cluster,Government operations and politics_ Government information and archives_cluster,Government operations and politics_ political campaign regulation_cluster,Government operations and politics_ Elections_cluster,Government operations and politics_ voting_cluster,Government operations and politics_ State and local government operations_cluster,Government operations and politics_ benefits_cluster,Government operations and politics_ personnel management_cluster,Government operations and politics_ Government employee pay_cluster,...,Health_ Health_cluster,Health_ Health programs administration and funding_cluster,Health_ Congressional oversight_cluster,Health_ Health promotion and preventive care_cluster,Health_ Prescription drugs_cluster,Health_ Health care costs and insurance_cluster,Health_ Department of Health and Human Services_cluster,Health_Administrative law and regulatory procedures_cluster,Health_ Government information and archives_cluster,Health_ Government studies and investigations_cluster
0,vote_A000374,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,0.0,...,2.0,2.0,2.0,1.0,2.0,0.0,1.0,0.0,0.0,1.0
1,vote_A000370,2.0,0.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,...,0.0,0.0,0.0,2.0,0.0,1.0,2.0,1.0,2.0,2.0
2,vote_A000055,0.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,...,1.0,2.0,2.0,1.0,1.0,0.0,0.0,2.0,0.0,1.0
3,vote_A000371,2.0,0.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,...,0.0,0.0,0.0,2.0,0.0,1.0,2.0,1.0,2.0,2.0
4,vote_A000372,0.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,...,1.0,2.0,2.0,1.0,1.0,0.0,0.0,2.0,0.0,1.0


In [63]:
clusters = cluster_df.columns
print(len(clusters))
print(clusters)

51
Index(['voters',
       'Government operations and politics_ Government operations and politics_cluster',
       'Government operations and politics_ Government information and archives_cluster',
       'Government operations and politics_ political campaign regulation_cluster',
       'Government operations and politics_ Elections_cluster',
       'Government operations and politics_ voting_cluster',
       'Government operations and politics_ State and local government operations_cluster',
       'Government operations and politics_ benefits_cluster',
       'Government operations and politics_ personnel management_cluster',
       'Government operations and politics_ Government employee pay_cluster',
       'Government operations and politics_ public corruption_cluster',
       'Finance and financial sector_ Finance and financial sector_cluster',
       'Finance and financial sector_ Government studies and investigations_cluster',
       'Finance and financial sector_ Congression

In [64]:
cluster_size_df = cluster_df.groupby(clusters[1]).size()
cluster_size_df = cluster_size_df.reset_index()
cluster_name = cluster_size_df.columns[0]
del cluster_size_df[cluster_name]
cluster_size_df = cluster_size_df.rename(columns={0:cluster_name})

In [65]:
cluster_size_df.head()

,Government operations and politics_ Government operations and politics_cluster
0,212
1,85
2,219


In [67]:
for c in clusters[2:]:
    cluster_sizes = cluster_df.groupby(c).size()
    cluster_sizes = cluster_sizes.reset_index()
    del cluster_sizes[c]
    cluster_sizes = cluster_sizes.rename(columns={0:c})
    cluster_size_df = pd.concat([cluster_size_df, cluster_sizes], axis=1)
    

In [70]:
cluster_size_df.to_csv("cluster_sizes.csv")